# LangChain Cookbook 👨‍🍳👩‍🍳

*This cookbook is based off the [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)*

**Goal:** Provide an introductory understanding of the components and use cases of LangChain via [ELI5](https://www.dictionary.com/e/slang/eli5/#:~:text=ELI5%20is%20short%20for%20%E2%80%9CExplain,a%20complicated%20question%20or%20problem.) examples and code snippets. For use cases check out [part 2](https://github.com/gkamradt/langchain-tutorials/blob/main/LangChain%20Cookbook%20Part%202%20-%20Use%20Cases.ipynb). See [video tutorial](https://www.youtube.com/watch?v=2xxziIWmaSA) of this notebook.


**Links:**
* [LC Conceptual Documentation](https://docs.langchain.com/docs/)
* [LC Python Documentation](https://python.langchain.com/en/latest/)
* [LC Javascript/Typescript Documentation](https://js.langchain.com/docs/)
* [LC Discord](https://discord.gg/6adMQxSpJS)
* [www.langchain.com](https://langchain.com/)
* [LC Twitter](https://twitter.com/LangChainAI)


### **What is LangChain?**
> LangChain is a framework for developing applications powered by language models.

**~~TL~~DR**: LangChain makes the complicated parts of working & building with AI models easier. It helps do this in two ways:

1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs
2. **Agency** - Allow your LLMs to interact with it's environment via decision making. Use LLMs to help decide which action to take next

### **Why LangChain?**
1. **Components** - LangChain makes it easy to swap out abstractions and components necessary to work with language models.

2. **Customized Chains** - LangChain provides out of the box support for using and customizing 'chains' - a series of actions strung together.

3. **Speed 🚢** - This team ships insanely fast. You'll be up to date with the latest LLM features.

4. **Community 👥** - Wonderful discord and community support, meet ups, hackathons, etc.

Though LLMs can be straightforward (text-in, text-out) you'll quickly run into friction points that LangChain helps with once you develop more complicated applications.

*Note: This cookbook will not cover all aspects of LangChain. It's contents have been curated to get you to building & impact as quick as possible. For more, please check out [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)*

*Update Oct '23: This notebook has been expanded from it's original form*

You'll need an OpenAI api key to follow this tutorial. You can have it as an environement variable, in an .env file where this jupyter notebook lives, or insert it below where 'YourAPIKey' is. Have if you have questions on this, put these instructions into [ChatGPT](https://chat.openai.com/).

In [92]:
from dotenv import load_dotenv
import os

load_dotenv()

azure_openai_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", None)
azure_openai_key=os.getenv("AZURE_OPENAI_API_KEY", None)
azure_api_version=os.getenv("AZURE_API_VERSION", None)
azure_deployment=os.getenv("AZURE_DEPLOYMENT", None)
azure_embedding_model=os.getenv("AZURE_EMBEDDING_MODEL", None)

def get_azure_chat_model() -> AzureChatOpenAI:
    return AzureChatOpenAI(
        openai_api_version=azure_api_version,
        azure_endpoint=azure_openai_endpoint,
        openai_api_key=azure_openai_key,
        azure_deployment=azure_deployment,
        temperature=0.7,
    )

def get_azure_embedding_model() -> AzureOpenAIEmbeddings:
    return AzureOpenAIEmbeddings(
        model=azure_embedding_model,
        azure_endpoint=azure_openai_endpoint,
        openai_api_key=azure_openai_key,
        api_version=azure_api_version,
    )

# LangChain Components

## Schema - Nuts and Bolts of working with Large Language Models (LLMs)

### **Text**
The natural language way to interact with LLMs

In [52]:
# You'll be working with simple strings (that'll soon grow in complexity!)
my_text = "What day comes after Friday?"
my_text

'What day comes after Friday?'

### **Chat Messages**
Like text, but specified with a message type (System, Human, AI)

* **System** - Helpful background context that tell the AI what to do
* **Human** - Messages that are intented to represent the user
* **AI** - Messages that show what the AI responded with

For more, see OpenAI's [documentation](https://platform.openai.com/docs/guides/chat/introduction)

In [53]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = get_azure_chat_model()

Now let's create a few messages that simulate a chat experience with a bot

In [54]:
ai_message = chat.invoke(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)
print(f"Response: {ai_message.content}")

Response: How about a fresh caprese salad with tomatoes, mozzarella, basil, and a drizzle of balsamic glaze?


You can also pass more chat history w/ responses from the AI

In [55]:
ai_message = chat.invoke(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)
print(f"Response: {ai_message.content}")

Response: Explore the charming Old Town and visit the beautiful Promenade des Anglais.


You can also exclude the system message if you want

In [56]:
ai_message = chat.invoke(
    [
        HumanMessage(content="What day comes after Thursday?")
    ]
)
print(f"Response: {ai_message.content}")

Response: The day that comes after Thursday is Friday.


### **Documents**
An object that holds a piece of text and metadata (more information about that text)

In [57]:
from langchain.schema import Document

In [58]:
Document(
    page_content="This is my document. It is full of text that I've gathered from other places",
    metadata={
        'my_document_id' : 234234,
        'my_document_source' : "The LangChain Papers",
        'my_document_create_time' : 1680013019
    }
)

Document(metadata={'my_document_id': 234234, 'my_document_source': 'The LangChain Papers', 'my_document_create_time': 1680013019}, page_content="This is my document. It is full of text that I've gathered from other places")

But you don't have to include metadata if you don't want to

In [59]:
Document(page_content="This is my document. It is full of text that I've gathered from other places")

Document(metadata={}, page_content="This is my document. It is full of text that I've gathered from other places")

## Models - The interface to the AI brains

###  **Language Model**
A model that does text in ➡️ text out!

*Check out how I changed the model I was using from the default one to ada-001 (a very cheap, low performing model). See more models [here](https://platform.openai.com/docs/models)*

### **Chat Model**
A model that takes a series of messages and returns a message output

In [60]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = get_azure_chat_model()

In [61]:
ai_message = chat.invoke(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)
print(f"Response: {ai_message.content}")

Response: Well, you could start by getting a plane ticket, but if you want to save money, just hitch a ride on a flying pigeon. They're the original frequent flyers!


### Function Calling Models

[Function calling models](https://openai.com/blog/function-calling-and-other-api-updates) are similar to Chat Models but with a little extra flavor. They are fine tuned to give structured data outputs.

This comes in handy when you're making an API call to an external service or doing extraction.

In [62]:
chat = get_azure_chat_model()

output = chat(messages=
     [
         SystemMessage(content="You are an helpful AI bot"),
         HumanMessage(content="What’s the weather like in Boston right now?")
     ],
     functions=[
         {
             "name": "get_current_weather",
             "description": "Get the current weather in a given location",
             "parameters": {
                 "type": "object",
                 "properties": {
                     "location": {
                         "type": "string",
                         "description": "The city and state, e.g. San Francisco, CA"
                     },
                     "unit": {
                         "type": "string",
                         "enum": ["celsius", "fahrenheit"]
                     }
                 },
                 "required": ["location"]
             }
         }
     ]
)
output

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"location":"Boston, MA"}', 'name': 'get_current_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 89, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'function_call', 'logprobs': None, 'content_filter_results': {}}, id='run--6d9ce171-4c54-4854-853f-ddad338ecd6e-0', usage_metadata={'input_tokens': 89, 'output_tokens'

See the extra `additional_kwargs` that is passed back to us? We can take that and pass it to an external API to get data. It saves the hassle of doing output parsing.

### **Text Embedding Model**
Change your text into a vector (a series of numbers that hold the semantic 'meaning' of your text). Mainly used when comparing two pieces of text together.

*BTW: Semantic means 'relating to meaning in language or logic.'*

In [63]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = get_azure_embedding_model()

In [64]:
text = "Hi! It's time for the beach"

In [65]:
text_embedding = embeddings.embed_query(text)
print (f"Here's a sample: {text_embedding[:5]}...")
print (f"Your embedding is length {len(text_embedding)}")

Here's a sample: [-0.02253468707203865, -0.016473732888698578, -0.003123833332210779, -0.04386293143033981, 0.014032116159796715]...
Your embedding is length 3072


## Prompts - Text generally used as instructions to your model

### **Prompt Template**
An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.

Think of it as an [f-string](https://realpython.com/python-f-strings/) in python but for prompts

In [66]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate

llm = get_azure_chat_model()


# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

template = PromptTemplate(
    input_variables=["location"],
    template=template,
)

chain = template | llm | StrOutputParser()
print(chain.invoke(input={"location": "Rome"}))

Explore the historic Colosseum, marvel at the Vatican City, and savor authentic Italian cuisine in Rome.


### **Example Selectors**
An easy way to select from a series of examples that allow you to dynamic place in-context information into your prompt. Often used when your task is nuanced or you have a large list of examples.

Check out different types of example selectors [here](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/)

If you want an overview on why examples are important (prompt engineering), check out [this video](https://www.youtube.com/watch?v=dOxUroR57xs)

In [79]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = get_azure_chat_model()
embeddings = get_azure_embedding_model()


example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings, 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma, 
    
    # This is the number of examples to produce.
    k=2
)

similar_prompt_template = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["noun"],
)

chain = similar_prompt_template | llm | StrOutputParser()
chain.invoke(input={"noun": "plant"})

'ground'

### **Output Parsers Method 1: Prompt Instructions & String Parsing**
A helpful way to format the output of a model. Usually used for structured output. LangChain has a bunch more output parsers listed on their [documentation](https://python.langchain.com/docs/modules/model_io/output_parsers).

Two big concepts:

**1. Format Instructions** - A autogenerated prompt that tells the LLM how to format it's response based off your desired result

**2. Parser** - A method which will extract your model's text output into a desired structure (usually json)

In [81]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [80]:
llm = get_azure_chat_model()

In [82]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [83]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [88]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt_template = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

chain = prompt_template | llm | output_parser
chain.invoke(input={"user_input": "welcom to califonya!"})


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to califonya!

YOUR RESPONSE:



{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

### **Output Parsers Method 2: OpenAI Fuctions**
When OpenAI released function calling, the game changed. This is recommended method when starting out.

They trained models specifically for outputing structured data. It became super easy to specify a Pydantic schema and get a structured output.

There are many ways to define your schema, I prefer using Pydantic Models because of how organized they are. Feel free to reference OpenAI's [documention](https://platform.openai.com/docs/guides/gpt/function-calling) for other methods.

In order to use this method you'll need to use a model that supports [function calling](https://openai.com/blog/function-calling-and-other-api-updates#:~:text=Developers%20can%20now%20describe%20functions%20to%20gpt%2D4%2D0613%20and%20gpt%2D3.5%2Dturbo%2D0613%2C). I'll use `gpt4-0613`

**Example 1: Simple**

Let's get started by defining a simple model for us to extract from.

In [98]:
from pydantic import BaseModel, Field
from typing import Optional, Sequence

class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")


Then let's create a chain (more on this later) that will do the extracting for us

In [103]:
from langchain.chains.openai_functions import create_structured_output_chain

llm = get_azure_chat_model()

prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="{user_input}"
)

chain = prompt_template | llm.with_structured_output(People)
chain.invoke(
    input={
        "user_input": "Sally is 43, Joey just turned 32 and loves spinach. Caroline is 10 years older than Sally."
    }
)

People(people=[Person(name='Sally', age=43, fav_food=None), Person(name='Joey', age=32, fav_food='spinach'), Person(name='Caroline', age=53, fav_food=None)])

Let's do some more parsing with it

**Example 2: Enum**

Now let's parse when a product from a list is mentioned

In [107]:
import enum

llm = get_azure_chat_model()

class Product(str, enum.Enum):
    CRM = "CRM"
    VIDEO_EDITING = "VIDEO_EDITING"
    HARDWARE = "HARDWARE"


class Products(BaseModel):
    """Identifying products that were mentioned in a text"""

    products: Sequence[Product] = Field(..., description="The products mentioned in a text")


In [108]:
chain = prompt_template | llm.with_structured_output(Products)
chain.invoke(
    input={
        "user_input": "The CRM in this demo is great. Love the hardware. The microphone is also cool. Love the video editing"
    }
)

Products(products=[<Product.CRM: 'CRM'>, <Product.HARDWARE: 'HARDWARE'>, <Product.VIDEO_EDITING: 'VIDEO_EDITING'>])

## Indexes - Structuring documents to LLMs can work with them

### **Document Loaders**
Easy ways to import data from other sources. Shared functionality with [OpenAI Plugins](https://openai.com/blog/chatgpt-plugins) [specifically retrieval plugins](https://github.com/openai/chatgpt-retrieval-plugin)

See a [big list](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) of document loaders here. A bunch more on [Llama Index](https://llamahub.ai/) as well.

**HackerNews**

In [109]:
from langchain.document_loaders import HNLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [110]:
loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

In [111]:
data = loader.load()

In [112]:
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman on Jan 18, 2023  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are veOzzie_osman on Jan 18, 2023  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_ind


**URLs and webpages**

Let's try it out with [Paul Graham's website](http://www.paulgraham.com/)

In [115]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "http://www.paulgraham.com/",
]

loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

data[0].page_content

[nltk_data] Error loading averaged_perceptron_tagger_eng: <urlopen
[nltk_data]     error [SSL: CERTIFICATE_VERIFY_FAILED] certificate
[nltk_data]     verify failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1006)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


'New: What to Do | Wokeness | Founder Mode Want to start a startup? Get funded by Y Combinator . © mmxxv pg'

### **Text Splitters**
Often times your document is too long (like a book) for your LLM. You need to split it up into chunks. Text splitters help with this.

There are many ways you could split your text into chunks, experiment with [different ones](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) to see which is best for you.

In [116]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [117]:
# This is a long document we can split up.
with open('data/PaulGrahamEssays/worked.txt') as f:
    pg_work = f.read()
    
print (f"You have {len([pg_work])} document")

You have 1 document


In [118]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 150,
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([pg_work])

In [119]:
print (f"You have {len(texts)} documents")

You have 610 documents


In [120]:
print ("Preview:")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

Preview:
February 2021Before college the two main things I worked on, outside of school,
were writing and programming. I didn't write essays. I wrote what 

beginning writers were supposed to write then, and probably still
are: short stories. My stories were awful. They had hardly any plot,


There are a ton of different ways to do text splitting and it really depends on your retrieval strategy and application design. Check out more splitters [here](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

### **Retrievers**
Easy way to combine documents with language models.

There are many different types of retrievers, the most widely supported is the VectoreStoreRetriever

In [121]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('data/PaulGrahamEssays/worked.txt')
documents = loader.load()

In [124]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = get_azure_embedding_model()

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [125]:
# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

In [126]:
retriever

VectorStoreRetriever(tags=['FAISS', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x32565d210>, search_kwargs={})

In [128]:
docs = retriever.invoke("what types of things did the author want to build?")

In [129]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

standards; what was the point? No one else wanted one either, so
off they went. That was what happened to systems work.I wanted not just to build things, but to build things that would
last.In this di

much of it in grad school.Computer Science is an uneasy alliance between two halves, theory
and systems. The theory people prove things, and the systems people
build things. I wanted to build things. 


### **VectorStores**
Databases to store vectors. Most popular ones are [Pinecone](https://www.pinecone.io/) & [Weaviate](https://weaviate.io/). More examples on OpenAIs [retriever documentation](https://github.com/openai/chatgpt-retrieval-plugin#choosing-a-vector-database). [Chroma](https://www.trychroma.com/) & [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) are easy to work with locally.

Conceptually, think of them as tables w/ a column for embeddings (vectors) and a column for metadata.

Example

| Embedding      | Metadata |
| ----------- | ----------- |
| [-0.00015641732898075134, -0.003165106289088726, ...]      | {'date' : '1/2/23}       |
| [-0.00035465431654651654, 1.4654131651654516546, ...]   | {'date' : '1/3/23}        |

In [130]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('data/PaulGrahamEssays/worked.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = get_azure_embedding_model()

In [131]:
print (f"You have {len(texts)} documents")

You have 78 documents


In [132]:
embedding_list = embeddings.embed_documents([text.page_content for text in texts])

In [133]:
print (f"You have {len(embedding_list)} embeddings")
print (f"Here's a sample of one: {embedding_list[0][:3]}...")

You have 78 embeddings
Here's a sample of one: [0.022806664928793907, -0.0013218469684943557, -0.023677930235862732]...


Your vectorstore store your embeddings (☝️) and make them easily searchable

## Memory
Helping LLMs remember information.

Memory is a bit of a loose term. It could be as simple as remembering information you've chatted about in the past or more complicated information retrieval.

We'll keep it towards the Chat Message use case. This would be used for chat bots.

There are many types of memory, explore [the documentation](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html) to see which one fits your use case.

### Chat Message History

In [134]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = get_azure_chat_model()

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of france?")

In [135]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, response_metadata={})]

In [137]:
ai_response = chat.invoke(history.messages)
ai_response

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 20, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered'

In [138]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={})]

## Chains ⛓️⛓️⛓️
Combining different LLM calls and action automatically

Ex: Summary #1, Summary #2, Summary #3 > Final Summary

Check out [this video](https://www.youtube.com/watch?v=f9_BWhCI4Zo&t=2s) explaining different summarization chain types

There are [many applications of chains](https://python.langchain.com/en/latest/modules/chains/how_to_guides.html) search to see which are best for your use case.

We'll cover two of them:

### Simple Sequential Chains

Easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks (and keeping your LLM focused)

In [139]:
from langchain.prompts import PromptTemplate

llm = get_azure_chat_model()

In [143]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = prompt_template | llm | StrOutputParser()

In [144]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = prompt_template | llm

In [145]:
overall_chain = location_chain | meal_chain

In [155]:
review = overall_chain.invoke(input={"user_location": "Rome"})
print(f"Response:\n{review.content}")

Response:
Cacio e Pepe Recipe

Ingredients:
- 12 oz (340 g) spaghetti or tonnarelli
- 1 cup (100 g) finely grated Pecorino Romano cheese
- 2 tsp freshly ground black pepper
- Salt, to taste

Instructions:
1. Bring a large pot of salted water to a boil. Add the pasta and cook until al dente, according to package instructions. Reserve 1 cup of pasta water and then drain the pasta.
2. In a large skillet over medium heat, toast the black pepper for about 1 minute until fragrant.
3. Add 1/2 cup of the reserved pasta water to the skillet and bring to a simmer.
4. Reduce the heat to low and add the cooked pasta to the skillet.
5. Gradually add the grated Pecorino Romano cheese, stirring quickly to create a creamy sauce. Add more pasta water as needed to achieve the desired consistency.
6. Toss the pasta until it is evenly coated with the cheese and pepper sauce.
7. Serve immediately, with extra Pecorino Romano cheese and black pepper on top if desired.

Enjoy your authentic Roman Cacio e Pepe